In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import joblib
from fastapi import FastAPI
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

import glob
import os

In [3]:
nltk.download('stopwords')

# Função para processar dados
def parse_lista(valor):
    return [item.strip() for item in str(valor).split(',')]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/brunomelo/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
user_path = "datathon_files/files/treino"
site_path = "datathon_files/itens/itens"
validacao_path = "datathon_files/validacao.csv"

In [8]:
site_files = glob.glob(os.path.join(site_path, "*.csv"))
site = pd.concat((pd.read_csv(f) for f in site_files), ignore_index=True)

In [9]:
user_files = glob.glob(os.path.join(user_path, "*.csv"))
usuarios = pd.concat((pd.read_csv(f) for f in user_files), ignore_index=True)

In [10]:
validacao = pd.read_csv(validacao_path)

In [11]:
# Processar usuários
def processar_usuarios(df):
    list_columns = ['history', 'timestampHistory', 'numberOfClicksHistory',
                   'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory']

    for col in list_columns:
        df[col] = df[col].apply(parse_lista)

    # Engenharia de features
    df['total_cliques'] = df['numberOfClicksHistory'].apply(lambda x: sum(map(int, x)))
    df['tempo_medio'] = df['timeOnPageHistory'].apply(lambda x: np.mean(list(map(int, x))))
    df['scroll_medio'] = df['scrollPercentageHistory'].apply(lambda x: np.mean(list(map(float, x))))
    df['visitas_total'] = df['pageVisitsCountHistory'].apply(lambda x: sum(map(int, x)))

    return df

usuarios_processados = processar_usuarios(usuarios)

In [12]:
# Processar conteúdo das notícias
site['conteudo'] = site['title'] + ' ' + site['caption']
site['conteudo'] = site['conteudo'].apply(lambda x: unidecode(x.lower()))
portuguese_stopwords = set(stopwords.words('portuguese'))
tfidf = TfidfVectorizer(stop_words=list(portuguese_stopwords))
tfidf_matrix = tfidf.fit_transform(site['conteudo'])

In [13]:
# Clusterização de usuários
features = usuarios_processados[['total_cliques', 'tempo_medio', 'scroll_medio', 'visitas_total']]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

kmeans = KMeans(n_clusters=5, random_state=42)
usuarios_processados['cluster'] = kmeans.fit_predict(features_scaled)



In [14]:
# Matriz de similaridade
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(site.index, index=site['page']).drop_duplicates()

MemoryError: Unable to allocate 140. GiB for an array with shape (18855907135,) and data type int64

In [ ]:
def validar_modelo(df_validacao, sistema):
    from sklearn.metrics import precision_at_k

    y_true = df_validacao['history'].apply(parse_lista)
    y_pred = [sistema.recomendar_hibrido(row['userId'], 10) for _, row in df_validacao.iterrows()]

    # Calcular precisão@k
    precision = []
    for true, pred in zip(y_true, y_pred):
        relevant = set(true)
        recommended = set(pred)
        precision.append(len(relevant & recommended) / len(recommended))

    return np.mean(precision)

# Testar precisão
print(f"Precisão@10: {validar_modelo(validacao, sistema):.2%}")

In [ ]:
class SistemaRecomendacao:
    def __init__(self, kmeans, tfidf, cosine_sim, scaler, usuarios, site):
        self.kmeans = kmeans
        self.tfidf = tfidf
        self.cosine_sim = cosine_sim
        self.scaler = scaler
        self.usuarios = usuarios
        self.site = site
        self.indices = pd.Series(site.index, index=site['Page'])

    def recomendar_hibrido(self, user_id, n=10):
        # ... (mesma lógica anterior)

    def salvar_modelo(self, caminho):
        joblib.dump({
            'kmeans': self.kmeans,
            'tfidf': self.tfidf,
            'cosine_sim': self.cosine_sim,
            'scaler': self.scaler,
            'usuarios': self.usuarios,
            'site': self.site
        }, caminho)

    @classmethod
    def carregar_modelo(cls, caminho):
        modelo = joblib.load(caminho)
        return cls(
            modelo['kmeans'],
            modelo['tfidf'],
            modelo['cosine_sim'],
            modelo['scaler'],
            modelo['usuarios'],
            modelo['site']
        )

# Salvar modelo treinado
sistema = SistemaRecomendacao(kmeans, tfidf, cosine_sim, scaler, usuarios_processados, site)
sistema.salvar_modelo('modelo_recomendacao.pkl')

In [ ]:
app = FastAPI()
modelo = None

@app.on_event("startup")
def carregar_modelo():
    global modelo
    modelo = SistemaRecomendacao.carregar_modelo('modelo_recomendacao.pkl')

@app.get("/recomendar/{user_id}")
def recomendar(user_id: str):
    recomendacoes = modelo.recomendar_hibrido(user_id, 10)
    return {"user_id": user_id, "recomendacoes": recomendacoes}

@app.post("/atualizar_modelo")
def atualizar_modelo(novo_caminho: str):
    global modelo
    modelo = SistemaRecomendacao.carregar_modelo(novo_caminho)
    return {"status": "modelo atualizado"}